<a href="https://colab.research.google.com/github/seunga2590/opensource_AI_python/blob/main/Deeplearning_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
# GPU 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# 4명의 데이터, 1명당 2개의 컬럼을 가짐 -> 2차원 리스트로 4명의 데이터를 선언 후, 토치의 텐서타입으로 바꿈. 이때, 텐서타입은 float
# to(device)는 모델이 GPU에서 연산될 것이기에 to(cuda)를 함으로써 데이터를 gpu에 얹혀 놓는다.
 
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [11]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), 
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True),
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True),
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True),
          nn.Sigmoid()
          ).to(device)

In [12]:
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [15]:
for epoch in range(50):
    optimizer.zero_grad() #옵티마이저의 기울기를 매번 초기화해서 계산해야하기 때문에 초기화하는 함수 선언
    hypothesis = model(X) # 모델에 데이터를 넣고, propagation을 진행한 후, 최종 아웃풋을 hypothesis에 담는다.

    cost = criterion(hypothesis, Y) #예측값과 정답값 사이의 Mean Square Loss를 구함
    cost.backward() #기울기를 구하는 함수
    optimizer.step() # backpropagation을 실행해 모델의 모든 w를 업데이트

    if epoch % 5 == 0:
        print("에폭:", epoch, cost.item()**(1/2)) # 에포크마다 cost값 출력 # 출력값 = RMSE

에폭: 0 0.4999190354359732
에폭: 5 0.4998889442133957
에폭: 10 0.4998179342669248
에폭: 15 0.4997455128100464
에폭: 20 0.4996222618892676
에폭: 25 0.4994388351890409
에폭: 30 0.4991480901198411
에폭: 35 0.49866937368065334
에폭: 40 0.4978595456141407
에폭: 45 0.4964364137451552


In [17]:
x_2 = torch.FloatTensor([[0, 0], [0, 1]]).to(device)
hypothesis = model(x_2)
print(hypothesis)

tensor([[0.4833],
        [0.5127]], device='cuda:0', grad_fn=<SigmoidBackward0>)


MINI BATCH DNN

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [21]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [24]:
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [33]:
model = nn.Sequential(
          nn.Linear(3, 10, bias=True), 
          nn.ReLU(),
          nn.Linear(10, 10, bias=True), 
          nn.ReLU(),
          nn.Linear(10, 10, bias=True), 
          nn.ReLU(),
          nn.Linear(10, 1, bias=True), 
          nn.ReLU()
          ).to(device)

In [34]:
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) # 0.00001

In [35]:
nb_epochs = 5
for epoch in range(nb_epochs + 1): 
  for batch_idx, samples in enumerate(dataloader): # mini batch사용 -> GPU가 부족해서

    x_train, y_train = samples

    x_train= x_train.to(device)
    y_train = y_train.to(device)
    
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = criterion(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))
    
    # 한 번에 데이터를 넣을 때와 나누어 넣을 때 나누어 넣으면 backpropagation을 더 많이하고 가중치 값이 더 많아져 overfitting을 유발할 수 있음
    # mini batch 할 경우, 에포크를 줄여서 진행

Epoch    0/5 Batch 1/3 Cost: 27752.000000
Epoch    0/5 Batch 2/3 Cost: 29290.000000
Epoch    0/5 Batch 3/3 Cost: 34225.000000
Epoch    1/5 Batch 1/3 Cost: 33312.500000
Epoch    1/5 Batch 2/3 Cost: 21634.000000
Epoch    1/5 Batch 3/3 Cost: 38416.000000
Epoch    2/5 Batch 1/3 Cost: 27194.500000
Epoch    2/5 Batch 2/3 Cost: 27752.000000
Epoch    2/5 Batch 3/3 Cost: 38416.000000
Epoch    3/5 Batch 1/3 Cost: 36320.500000
Epoch    3/5 Batch 2/3 Cost: 26282.000000
Epoch    3/5 Batch 3/3 Cost: 23104.000000
Epoch    4/5 Batch 1/3 Cost: 30760.000000
Epoch    4/5 Batch 2/3 Cost: 33312.500000
Epoch    4/5 Batch 3/3 Cost: 20164.000000
Epoch    5/5 Batch 1/3 Cost: 21634.000000
Epoch    5/5 Batch 2/3 Cost: 35408.000000
Epoch    5/5 Batch 3/3 Cost: 34225.000000


### CNN

In [36]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(24)

In [38]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [39]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 458999450.69it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 119345511.16it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 170094970.28it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23724195.23it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [40]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [41]:
# torch.nn.Module:  PyTorch의 모든 Neural Network의 Base Class
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [42]:
# CNN 모델 정의
model = CNN().to(device)

In [43]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [45]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape

        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.212304473
[Epoch:    2] cost = 0.060716711
[Epoch:    3] cost = 0.0448936783
[Epoch:    4] cost = 0.036229115
[Epoch:    5] cost = 0.0306792744
[Epoch:    6] cost = 0.0251198597
[Epoch:    7] cost = 0.021560574
[Epoch:    8] cost = 0.0185507275
[Epoch:    9] cost = 0.0156751964
[Epoch:   10] cost = 0.0137288449
[Epoch:   11] cost = 0.0105320672
[Epoch:   12] cost = 0.0106347837
[Epoch:   13] cost = 0.00774629833
[Epoch:   14] cost = 0.00656339806
[Epoch:   15] cost = 0.00614164583


In [47]:
from sklearn.metrics import f1_score

# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 
                        # 이를 axis 1방향으로 max값을 찾는다는 것
    correct_prediction = torch.argmax(prediction, 1) == Y_test #  Y_test는 숫자 정답값임.
                          # torch.argmax(prediction, 1)는 피쳐값중 max인 인덱스(=label)을 뽑는것 
    
    # print( f1_score(torch.argmax(prediction, 1),Y_test) )
     
    #accuracy = correct_prediction.float().mean()
    #print('Accuracy:', accuracy.item())

/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [48]:
# 모델 저장 
torch.save(model.state_dict(),"cnn_model.pt")

In [49]:
model = CNN().to(device)
model.load_state_dict(torch.load('cnn_model.pt'))
model.eval()

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [50]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
  
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.988599956035614


/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [52]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

with torch.no_grad():

    # 이미지 파일 경로 설정
    img = Image.open("./8.png")
    transform = transforms.Compose([
          transforms.Grayscale(num_output_channels=1), # RGB(3D) -> Gray(2D)
          transforms.Resize((28, 28)), # 모델 인풋에 맞게
          transforms.ToTensor(), # 토치 텐서 타입으로 맞춰줘야함
      ])       
    
    img_tensor = transform(img).to(device) # [1, 28, 28]
    img_tensor = img_tensor.unsqueeze(0) # [1, 1, 28, 28] # [1, 3, 28, 28]이었다면 RGB타입

    print(img_tensor.shape)

    prediction = model(img_tensor)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것 
    print('result:', torch.argmax(prediction, 1))
    print(prediction) # 10개의 아웃풋 중 가장 큰 값을 찾기

torch.Size([1, 1, 28, 28])
result: tensor([8], device='cuda:0')
tensor([[ -8.7651, -16.2537,  -4.8365,   0.3974, -10.5339, -10.0879, -12.2443,
         -11.2590,  17.7824,   2.1562]], device='cuda:0')
